# Creating Bike and Pedestrian Road Links

For environment see: https://github.com/gboeing/osmnx 

    conda config --prepend channels conda-forge
    
    conda create -n ox --strict-channel-priority osmnx
    
* also include nb_conda, nb_conda_kernels, and ipykernel

In [1]:
import geopandas as gpd
import osmnx
import pandas as pd
import networkx as nx

In [5]:
#import TAZs
tazs = gpd.read_file('C:/Users/phoebe/Bike_Ped_Skims_OSM/candidate_CTPS_TAZ_STATEWIDE_2019.geojson')
#set the projected crs
tazs = tazs.to_crs('EPSG:26986')
#boundaries for TAZ
states = tazs.dissolve()

#update the attributes retrieved from OSM in settings.py
useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 
                    'surface', 'smoothness','cycleway', 'footway', 'sidewalk']
osmnx.utils.config(useful_tags_way = useful_tags_path)

utn = osmnx.settings.useful_tags_node
oxna = osmnx.settings.osm_xml_node_attrs
oxnt = osmnx.settings.osm_xml_node_tags
utw = osmnx.settings.useful_tags_way
oxwa = osmnx.settings.osm_xml_way_attrs
oxwt = osmnx.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
osmnx.config(all_oneway=True, useful_tags_node=utn, useful_tags_way=utw)

#grab from OSM the bike and pedestrian networks for the MPO
#WARNING: Takes 48 hours to run (for both bike and ped together)
bike_net = osmnx.graph_from_polygon(states.to_crs('EPSG:4326').unary_union, network_type='bike')
ped_net = osmnx.graph_from_polygon(states.to_crs('EPSG:4326').unary_union, network_type='walk')


In [ ]:
#update the attributes retrieved from OSM in settings.py
useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 
                    'surface', 'smoothness','cycleway', 'footway', 'sidewalk']
osmnx.utils.config(useful_tags_way = useful_tags_path)

utn = osmnx.settings.useful_tags_node
oxna = osmnx.settings.osm_xml_node_attrs
oxnt = osmnx.settings.osm_xml_node_tags
utw = osmnx.settings.useful_tags_way
oxwa = osmnx.settings.osm_xml_way_attrs
oxwt = osmnx.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
osmnx.config(all_oneway=True, useful_tags_node=utn, useful_tags_way=utw)

#grab from OSM the bike and pedestrian networks for the MPO
#WARNING: Takes 48 hours to run (for both bike and ped together)
place = ['Massachusetts', 'Rhode Island', 'New Hampshire']
bike_net = osmnx.graph_from_place(place, network_type='bike')
ped_net = osmnx.graph_from_place(place, network_type='walk')


In [ ]:
osmnx.io.save_graph_shapefile(bike_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Bike')

In [ ]:
osmnx.io.save_graph_shapefile(ped_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Ped')

In [8]:
states

,geometry,town,OBJECTID,mpo,taz,state,town_state,in_brmpo,Shape_Leng,type,id,subregion
0,"MULTIPOLYGON (((195473.042 769042.293, 195474....",MIDDLEBOROUGH,1,SRPEDD,0,MA,"MIDDLEBOROUGH,MA",0,22284.463348,I,4398,SSC
